In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
import os


# approach

### Input last exons - selecting representative PAS/last exon:

le_ids can have multiple predictions, need to select a single one.
Strategy:
(starting with novel last exons used for quantification) 
1. Prefer events that match atlas
2. if multiple, select min atlas distance
3. If matches, s





## To-dos:
- fix updating to atlas for -ve strand matching pas
- run selection for all novel 3'UTR extensions
- Implement selection for annotated proximal sites of 3'UTR extensions (i.e. most distal annotated site)


# 

In [2]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-05-24_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")
# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")


In [3]:
print(quant_uniq_le.columns)
quant_uniq_le.subset(lambda df: ~df.transcript_id.str.startswith("ENST"))

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'transcript_id', 'exon_number',
       'region_rank', 'Start_ref', 'End_ref', 'transcript_id_ref',
       '3p_extension_length', 'event_type', 'ref_gene_id', 'ref_gene_name',
       'le_number', 'le_id'],
      dtype='object')


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24476667,.,+,.,PAPA.NT_19074713_S59.398,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
1,chr1,.,exon,24468985,24476667,.,+,.,PAPA.ctrl_ctrl_3.424,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
2,chr1,.,exon,24468985,24475256,.,+,.,PAPA.TDP43_19065409_S29.438,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
3,chr1,.,exon,24468985,24473773,.,+,.,PAPA.Cont-B_S2.334,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
4,chr1,.,exon,24468985,24474852,.,+,.,PAPA.DZ_curves_1_3.360,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36889,chrY,.,exon,13234576,13234826,.,-,.,PAPA.NT_19074729_S12.25635,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36890,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43-F_S6.20683,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36891,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43_19065413_S19.26168,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36892,chrY,.,exon,13234583,13234826,.,-,.,PAPA.DZ_curves_0_2.21889,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7


In [4]:
dexseq_df.columns

Index(['experiment_name', 'binID', 'groupID', 'featureID', 'exonBaseMean',
       'dispersion', 'stat', 'pvalue', 'padj', 'UsageCoefficient_base',
       'UsageCoefficient_contrast', 'log2fold_contrast_base', 'le_id',
       'gene.qvalue', 'contrast_name', 'gene_id', 'gene_name', 'event_type',
       'annot_status', 'transcript_id', 'chromosome', 'strand', 'start', 'end',
       'mean_PPAU_base', 'mean_PPAU_treatment', 'delta_PPAU_treatment_control',
       'simple_event_type'],
      dtype='object')

In [5]:
novel_le.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU'],
      dtype='object')

In [6]:
# get le_ids that are novel 3'UTR ectensions
le_id_d3utr = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
len(le_id_d3utr)

3369

In [7]:
def n_uniq_coords(df):

    if df["Strand"].unique()[0] == "+":
        n_coords = df["End"].nunique()
    
    else:
        n_coords = df["Start"].nunique()

    return n_coords


def select_rep_site(df):
# First select sites of le_id that match atlas site (if applicable)
    if df["atlas_filter"].astype(int).sum() > 0:

        out_df = df[df["atlas_filter"].eq("1")]

        # 2. Within these, see if multiple different 3'ends predicted
        ### TODO: use Name (atlas ID) instead of 3' coordinate to match
        n_atlas = out_df["Name"].nunique()

        if n_atlas > 1:
                # multiple different atlas sites predicted
                # see if somes sites were called in > 1 dataset - select those with max number of datasets called
                end3_c = out_df.groupby("Name")["experiment_id"].nunique()
                # have a series with Name (PolyASite ID) as index labels, count of experimnts as values
                # get the IDs with largest number of unqiue experiments (as as series)
                end3_c = end3_c[end3_c == end3_c.max()]

                if len(end3_c) > 1:
                    # multiple 3'ends/ atlas sites predicted in same number of datasets
                    # raise Exception(f"{out_df['le_id_quant'].drop_duplicates().iloc[0]} - le_id has multiple predicted atlas sites")
                    # prefer the shortest extension in this case (conservative, defines smaller window in)

                    # calculate length
                    out_df = out_df[out_df["Name"].isin(end3_c.index)]
                    out_df["reg_len"] = out_df.End - out_df.Start

                    # select shortest extension
                    out_df = out_df[out_df.reg_len == out_df.reg_len.min()]
                    # Select a single row for le_id corresponding to selected PAS
                    out_df = out_df.sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"]).drop(columns="reg_len")

                    return out_df, "atlas_max_datasets_shortest"
                                    
                else:
                    # 1 pas remaining
                    end3_coord = end3_c.index[0]

                    # Select a single row for le_id corresponding to selected PAS
                    out_df = out_df[out_df["Name"] == end3_coord].sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"])

                    decision_str = "atlas_max_datasets"


        else:
            # only single atlas site predicted - just select a single row
            end3_coord = out_df["Name"].drop_duplicates().index[0]
            # Select a single row for le_id corresponding to selected PAS
            out_df = out_df[out_df["Name"] == end3_coord].sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"])

            decision_str = "atlas_1_pred"
        

        return out_df, decision_str

    else:
        # TODO: implement filtering for events called with motifs only
        out_df = df[df["atlas_filter"].eq("0")]

        # select predicted site with motif that is closest to expected distance (21nt upstream)
        # -74_TATAAA,-14_TATAAA
        # [-74_TATAAA, -14_TATAAA]
        # [-74, -14]
        # absolute distance from expected position (21nt upstream)
        # select min deviating motif for each predicted PAS
        pas_m = out_df["pas_motifs"].apply(lambda x: min([abs(21 - abs(int(i.split("_")[0]))) for i in x.split(",")]))

        # select predicted pas with minimum deviation from 21nt
        pas_m = pas_m[pas_m == pas_m.min()]

        if len(pas_m) > 1:
            # multiple predicted PAS with motif same distance from expected

            # check if exactly the same coordinate predicted for these events

            # (if true, just sort alphabetically by experiment id and pick first)
            # otherwise, select shortest
            n_pas = n_uniq_coords(out_df)

            if n_pas > 1:
                # select shortest extension
                # calculate length
                out_df = out_df.loc[pas_m.index, :]

                out_df["reg_len"] = out_df.End - out_df.Start

                # select shortest extension
                out_df = out_df[out_df.reg_len == out_df.reg_len.min()]
                # Select a single row for le_id corresponding to selected PAS
                out_df = out_df.sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"]).drop(columns="reg_len")

            else:
                # just a single coord predicted in multiple datasets - 
                out_df = out_df.sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"])

            return out_df, "motif_shortest_min"
        
        else:
            out_df = out_df.loc[pas_m.index, :]
            

            decision_str = "motif_1_min"


        return out_df, decision_str


    
    

In [8]:
#### SELECTING REPRESENTATIVE PAS
# LEs have to be quantified
novel_le_df = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"])).as_df()


i = 0

sel_dfs = []
sel_dict = {"atlas_1_pred": [],
            "atlas_max_datasets": [],
            "atlas_max_datasets_shortest": [],
            "motif_1_min": [],
            "motif_shortest_min": []}

le_grpd = iter(novel_le_df[novel_le_df.le_id.isin(le_id_d3utr)].groupby("le_id_quant"))



while i < 100:
        le_id, df = next(le_grpd)
        # print(le_id)
        out_df, decision_str = select_rep_site(df)
        sel_dfs.append(out_df)
        sel_dict[decision_str].append(le_id)
        i +=1  



sel_novel_le_df = pd.concat(sel_dfs)               
    
print({dec: len(ids) for dec, ids in sel_dict.items()})


{'atlas_1_pred': 41, 'atlas_max_datasets': 20, 'atlas_max_datasets_shortest': 8, 'motif_1_min': 29, 'motif_shortest_min': 2}


In [9]:
# novel_le_df[novel_le_df["le_id_quant"].isin(sel_dict["atlas_multiple_sites"])][["le_id_quant", "experiment_id", "Name", "Start", "End", "experiment_count", "transcript_id", "Strand"]].loc[lambda df: df.le_id_quant == ]

In [10]:
sel_novel_le_df[sel_novel_le_df["le_id_quant"].isin(sel_dict["atlas_max_datasets_shortest"])][["le_id_quant", "experiment_id", "Name", "Start", "End", "experiment_count", "transcript_id", "Strand"]]
#.loc[lambda df: df.le_id_quant == "ENSG00000006757.12_2", :]

,le_id_quant,experiment_id,Name,Start,End,experiment_count,transcript_id,Strand
13794,ENSG00000001629.10_3,zanovello_i3_cortical_upf1_tdp_tdpkd_upf1ctl_v...,7:92401774:+,92398211,92401760,6,PAPA.TDP43_ctrl_2.10802.2,+
14443,ENSG00000005108.17_3,zanovello_shsy5y_curve_0075,7:11370437:-,11370452,11375878,7,PAPA.doxconc_NT_0_2.7361.1,-
14450,ENSG00000006468.14_4,humphrey_i3_cortical,7:13890964:-,13890946,13896087,3,PAPA.TDP-3.9905.1,-
33381,ENSG00000006757.12_2,seddighi_i3_cortical,X:7898247:-,7898241,7900817,2,PAPA.NT_19074721_S50.25909.1,-
13230,ENSG00000008018.9_2,seddighi_i3_cortical,6:170534538:-,170534537,170535405,3,PAPA.TDP43_19065409_S29.9994.1,-
32454,ENSG00000015475.19_2,humphrey_i3_cortical,22:17733561:-,17733560,17735591,4,PAPA.CTRL-5.21657.1,-
21869,ENSG00000018236.15_4,brown_i3_cortical,12:41073374:+,41069958,41073383,6,PAPA.TDP43-F_S6.12521.4,+
764,ENSG00000031698.13_3,zanovello_shsy5y_chx_kd_only,1:109238791:+,109237730,109238792,2,PAPA.chx_tdp_CTRL_ctrl_4.997.3,+


In [11]:
print(novel_le_df[novel_le_df["le_id_quant"] == "ENSG00000001631.17_3"].columns)
novel_le_df[novel_le_df["le_id_quant"] == "ENSG00000001631.17_3"][["Start", "End", "atlas_filter", "Name", "le_id", "gene_name_ref"]]

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU',
       'le_id_quant'],
      dtype='object')


,Start,End,atlas_filter,Name,le_id,gene_name_ref
14815,92198195,92200804,0,7:92197498:-,ENSG00000001631.17_3,KRIT1
14816,92198602,92200804,0,7:92199243:-,ENSG00000001631.17_3,KRIT1
14817,92199253,92200804,1,7:92199243:-,ENSG00000001631.17_3,KRIT1
14818,92199257,92200804,1,7:92199243:-,ENSG00000001631.17_3,KRIT1


In [12]:
# now for all 3'utr extensions
sel_dfs_d3utr = []
sel_dict_d3utr = {"atlas_1_pred": [],
                  "atlas_max_datasets": [],
                  "atlas_max_datasets_shortest": [],
                  "motif_1_min": [],
                  "motif_shortest_min": []}


for le_id, df in novel_le_df[novel_le_df.le_id.isin(le_id_d3utr)].groupby("le_id_quant"):
        out_df, decision_str = select_rep_site(df)
        sel_dfs_d3utr.append(out_df)
        sel_dict_d3utr[decision_str].append(le_id)


sel_novel_le_df_d3utr = pd.concat(sel_dfs_d3utr)  
sel_novel_le_d3utr = pr.PyRanges(sel_novel_le_df_d3utr)             
    
print({dec: len(ids) for dec, ids in sel_dict_d3utr.items()})

{'atlas_1_pred': 1511, 'atlas_max_datasets': 437, 'atlas_max_datasets_shortest': 315, 'motif_1_min': 849, 'motif_shortest_min': 56}


In [13]:
# now need to select representative proximal site for each distal site, since quantification approach lumps together all annotated proximal sites (and compares them to extension)
# since le_ids are annotated sequentially, know that immediately preceding le_number is the partner le_id
d3utr_le_id_split = sel_novel_le_df_d3utr.le_id.str.split("_", regex=False, expand=True)
print(d3utr_le_id_split)
d3utr_le_id_split[1] = d3utr_le_id_split[1].astype(int).subtract(1)

# reconstruct le_id (corresponding to proximal le_id)
d3utr_prox_le_id = d3utr_le_id_split[0].str.cat(d3utr_le_id_split[1].astype(str), sep="_")
d3utr_prox_le_id

                        0  1
33668  ENSG00000000003.15  2
33669  ENSG00000000003.15  2
31647  ENSG00000000419.14  2
113    ENSG00000001461.17  6
13794  ENSG00000001629.10  3
...                   ... ..
31482   ENSG00000285230.3  2
33348   ENSG00000285756.2  4
2663    ENSG00000286019.1  2
29844   ENSG00000286104.1  4
941     ENSG00000286219.2  2

[2196 rows x 2 columns]


33668    ENSG00000000003.15_1
33669    ENSG00000000003.15_1
31647    ENSG00000000419.14_1
113      ENSG00000001461.17_5
13794    ENSG00000001629.10_2
                 ...         
31482     ENSG00000285230.3_1
33348     ENSG00000285756.2_3
2663      ENSG00000286019.1_1
29844     ENSG00000286104.1_3
941       ENSG00000286219.2_1
Name: 0, Length: 2196, dtype: object

In [14]:
#subset for annotated last exons with 3'end proximal to 3'UTR extensions
quant_uniq_le_d3utr_prox = quant_uniq_le.subset(lambda df: df.le_id.isin(set(d3utr_prox_le_id.values)))
quant_uniq_le_d3utr_prox


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,109237730,109238182,.,+,.,ENSG00000031698.13,SARS1,...,last,109237373,109237730,ENST00000468588.1,NULL,first_exon_spliced,ENSG00000031698.13,SARS1,2.0,ENSG00000031698.13_2
3,chr1,.,exon,109237730,109238169,.,+,.,ENSG00000031698.13,SARS1,...,last,109237373,109237730,ENST00000468588.1,NULL,first_exon_spliced,ENSG00000031698.13,SARS1,2.0,ENSG00000031698.13_2
4,chr1,.,exon,210674287,210676290,.,+,.,ENSG00000054392.13,HHAT,...,last,"210623670,210623670,210623670,210623670,210623...","210674287,210674287,210674287,210674287,210674...","ENST00000261458.8,ENST00000367009.2,ENST000003...","NULL,NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000054392.13,HHAT,2.0,ENSG00000054392.13_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6126,chrX,.,exon,3853009,3854927,.,-,.,ENSG00000285756.2,ENSG00000285756,...,last,3854927,3860789,ENST00000381108.3,NULL,last_exon_spliced,ENSG00000285756.2,ENSG00000285756,3.0,ENSG00000285756.2_3
6127,chrX,.,exon,3854348,3854927,.,-,.,ENSG00000285756.2,ENSG00000285756,...,last,3854927,3866921,ENST00000456563.1,NULL,first_exon_spliced,ENSG00000285756.2,ENSG00000285756,3.0,ENSG00000285756.2_3
6128,chrY,.,exon,2866792,2867268,.,+,.,ENSG00000129824.16,RPS4Y1,...,last,"2865245,2865245","2866792,2866792","ENST00000250784.13,ENST00000477725.1","NULL,NULL",last_exon_spliced,ENSG00000129824.16,RPS4Y1,1.0,ENSG00000129824.16_1
6129,chrY,.,exon,2866792,2866956,.,+,.,ENSG00000129824.16,RPS4Y1,...,last,"2865245,2865245","2866792,2866792","ENST00000250784.13,ENST00000477725.1","NULL,NULL",last_exon_spliced,ENSG00000129824.16,RPS4Y1,1.0,ENSG00000129824.16_1


In [15]:
# Since proximal_le_id collapses any annotated last exon into a single isoform, will need to select a representative proximal polyA site
# will arbitrarily select the most distal annotated site (in case of multiple)

def select_rep_prox_site(df: pd.DataFrame):
    '''_summary_

    Parameters
    ----------
    df : _type_
        _description_
    '''

    if (df.Strand == "+").all():
        # select the row with most distal 3'end (largest End coordinate)
        return df[df["End"] == df["End"].max()]
    
    elif (df.Strand == "-").all():
        # select the row with most distal 3'end (smallest Start coordinate)
        return df[df["Start"] == df["Start"].min()]
    
    else:
        raise ValueError("Strand column values must be all '+' or '-'")
    


In [16]:
sel_le_d3utr_prox = quant_uniq_le_d3utr_prox.apply(lambda df: df.groupby("le_id").apply(select_rep_prox_site).reset_index(drop=True))
sel_le_d3utr_prox

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,109237730,109238182,.,+,.,ENSG00000031698.13,SARS1,...,last,109237373,109237730,ENST00000468588.1,NULL,first_exon_spliced,ENSG00000031698.13,SARS1,2.0,ENSG00000031698.13_2
3,chr1,.,exon,210674287,210676296,.,+,.,ENSG00000054392.13,HHAT,...,last,"210623670,210623670,210623670,210623670,210623...","210674287,210674287,210674287,210674287,210674...","ENST00000261458.8,ENST00000367009.2,ENST000003...","NULL,NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000054392.13,HHAT,2.0,ENSG00000054392.13_2
4,chr1,.,exon,210674287,210676296,.,+,.,ENSG00000054392.13,HHAT,...,last,"210623670,210623670,210623670,210623670,210623...","210674287,210674287,210674287,210674287,210674...","ENST00000261458.8,ENST00000367009.2,ENST000003...","NULL,NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000054392.13,HHAT,2.0,ENSG00000054392.13_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,chrX,.,exon,153587595,153588394,.,-,.,PAPA.TDP43-E_S5.22033,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000262919.8,CCNQ,1.0,ENSG00000262919.8_1
2078,chrX,.,exon,153587595,153588394,.,-,.,PAPA.TDP43-E_S5.22033,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000262919.8,"CCNQ,CCNQ,CCNQ,CCNQ",1.0,ENSG00000262919.8_1
2079,chrX,.,exon,3853009,3854927,.,-,.,ENSG00000285756.2,ENSG00000285756,...,last,3854927,3860789,ENST00000381108.3,NULL,last_exon_spliced,ENSG00000285756.2,ENSG00000285756,3.0,ENSG00000285756.2_3
2080,chrY,.,exon,2866792,2867268,.,+,.,ENSG00000129824.16,RPS4Y1,...,last,"2865245,2865245","2866792,2866792","ENST00000250784.13,ENST00000477725.1","NULL,NULL",last_exon_spliced,ENSG00000129824.16,RPS4Y1,1.0,ENSG00000129824.16_1


In [17]:
# now want to combine the two with minimal info required for making maps
# 3'end coordinates, gene name, le_id, site type, regulation status

# define site_type columns
sel_le_d3utr_prox.site_type = "proximal"
sel_novel_le_d3utr.site_type = "distal"

# define cryptic le_ids
# define cryptic gene_ids
dexseq_df_cryp = dexseq_df[(dexseq_df["padj"].lt(0.05)) & (dexseq_df["mean_PPAU_base"].le(0.1)) & (dexseq_df["delta_PPAU_treatment_control"].ge(0.1))]
cryp_le_ids = set(dexseq_df_cryp.le_id)
cryp_gene_names = set(dexseq_df_cryp.gene_name)

# background - ns in any dataset (to do this, get list of sig in any dataset)
# get gene IDs with no regulated ALEs in any dataset
# returns pd.Series
ns_gene_ids = (dexseq_df.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
               .groupby("groupID")
               ["reg_status"]
               .sum()
               .loc[lambda x: x == 0]
             )

# if one isoform of pair is called as cryptic, label both as cryptic
cryp_gene_ids_d3utr = (pd.concat([sel_le_d3utr_prox.as_df()[["le_id", "gene_id"]].drop_duplicates(),
           sel_novel_le_d3utr.as_df()[["le_id", "gene_id_ref"]].rename(columns={"gene_id_ref": "gene_id"}).drop_duplicates()],
           ignore_index=True)
           .assign(cryptic_status=lambda df: np.where(df["le_id"].isin(cryp_le_ids), 1, 0))
           # for each gene_name, if any isoforms (prox/distal are called as cryptic, annotated as cryptic)
           .groupby("gene_id")
           ["cryptic_status"].sum()
           .loc[lambda x: x > 0]
)

ns_gene_ids_d3utr = (pd.concat([sel_le_d3utr_prox.as_df()[["le_id", "gene_id"]].drop_duplicates(),
           sel_novel_le_d3utr.as_df()[["le_id", "gene_id_ref"]].rename(columns={"gene_id_ref": "gene_id"}).drop_duplicates()],
           ignore_index=True)
           .assign(cryptic_status=lambda df: np.where(df["gene_id"].isin(ns_gene_ids.index), 1, 0))
           # for each gene_name, if any isoforms (prox/distal are called as cryptic, annotated as cryptic)
           .groupby("gene_id")
           ["cryptic_status"].sum()
           .loc[lambda x: x > 0]
)


print(f"Number of cryptic 3'UTR extension-containing genes - {len(cryp_gene_ids_d3utr)}")
print(f"Number of ns 3'UTR extension-containing genes - {len(ns_gene_ids_d3utr)}")




# sel_le_d3utr_prox
# sel_novel_le_d3utr




Number of cryptic 3'UTR extension-containing genes - 66
Number of ns 3'UTR extension-containing genes - 811


In [18]:
def _define_cryptic_status(df, cryptic_ids: iter, ns_ids: iter, id_col: str) -> pd.Series:
    '''_summary_

    Parameters
    ----------
    df : _type_
        _description_
    cryptic_ids : iter
        _description_
    ns_ids : iter
        _description_
    id_col : str
        _description_

    Returns
    -------
    pd.Series
        _description_
    '''

    assert id_col in df.columns

    choices = ["cryptic", "background"]
    conds = [df[id_col].isin(cryptic_ids), # in cryptic list
             ~df[id_col].isin(cryptic_ids) & df[id_col].isin(ns_ids) # not in cryptic list & in ns list
             ]
    
    return pd.Series(np.select(conds, choices, default="NULL"), index=df.index)


sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_gene_ids_d3utr.index, ns_gene_ids_d3utr.index, id_col="gene_id"))
sel_novel_le_d3utr = sel_novel_le_d3utr.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_gene_ids_d3utr.index, ns_gene_ids_d3utr.index, id_col="gene_id_ref"))



# sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))
# sel_novel_le_d3utr = sel_novel_le_d3utr.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id_ref.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))

In [19]:
# Construct 'name' string with minimal annotation information
# le_id|gene_name|site_type|cryptic_status

sel_le_d3utr_prox = (sel_le_d3utr_prox
                     .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("ref_gene_name",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("gene_name_common",
                          lambda df: pd.Series(np.where(df["gene_name"].isna(),
                                                        df["ref_gene_name"],
                                                        df["gene_name"])))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_common", "site_type", "cryptic_status"]], sep="|"))
                    
 )


sel_novel_le_d3utr = (sel_novel_le_d3utr
                      .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("Name",
         lambda df: df["le_id"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)


In [20]:
# generate output bed file of proximal and distal PAS
utr3_out_bed = pr.concat([sel_le_d3utr_prox[["Score", "Name"]],
                          sel_novel_le_d3utr[["Score", "Name"]]]
                          )

print(len(utr3_out_bed))
# drop duplicate entries
utr3_out_bed = utr3_out_bed.apply(lambda df: df.drop_duplicates()).sort()

utr3_out_bed

2163


,Chromosome,Start,End,Score,Strand,Name
0,chr1,24468985,24472976,.,+,ENSG00000001461.17_5|NIPAL3|proximal|background
1,chr1,24468985,24476667,.,+,ENSG00000001461.17_6|NIPAL3|distal|background
2,chr1,28496533,28500572,.,+,ENSG00000204138.13_3|PHACTR4|distal|background
3,chr1,28497271,28500364,.,+,ENSG00000204138.13_2|PHACTR4|proximal|background
4,chr1,28499419,28500364,.,+,ENSG00000204138.13_2|PHACTR4|proximal|background
...,...,...,...,...,...,...
1989,chrX,155276210,155278064,.,-,ENSG00000155962.13_1|CLIC2|proximal|background
1990,chrY,2866792,2867268,.,+,ENSG00000129824.16_1|RPS4Y1|proximal|background
1991,chrY,2866792,2867772,.,+,ENSG00000129824.16_2|RPS4Y1|distal|background
1992,chrY,13705224,13706024,.,+,ENSG00000154620.6_1|TMSB4Y|proximal|background


In [21]:
if not os.path.exists("../processed/iclip_regions"):
    os.makedirs("../processed/iclip_regions")
    
utr3_out_bed.to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_d3utr.last_exons.bed")
# also make one with just polYA sites
utr3_out_bed.three_end().to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_d3utr.pas.bed")